In [1]:
import syft as sy
from syft.lib import python
sy.VERBOSE = False
from syft.core.common.uid import UID

/Users/madhavajay/.local/share/virtualenvs/PySyft-lHlz_cKe/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py:125: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


In [2]:
alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_root_client()
alice.root_verify_key = alice_client.verify_key  # inject 📡🔑 as 📍🗝

In [3]:
# original MNIST imports
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision import datasets, transforms
# from torch.optim.lr_scheduler import StepLR

# get imports from remote client to match
torch = alice_client.torch
torchvision = alice_client.torchvision
transforms = torchvision.transforms
datasets = torchvision.datasets
nn = torch.nn
F = torch.nn.functional
optim = torch.optim
StepLR = torch.optim.lr_scheduler.StepLR
Net = alice_client.models.MNIST

In [4]:
# helper to create and send a primitive
def make(thing):
    prim = python.primitive_factory.PrimitiveFactory.generate_primitive(value=thing, recurse=True)
    ptr = prim.send(alice_client)
    return ptr

In [5]:
print(alice.store.peek)

{}


In [6]:
model = Net()
print(alice.store.peek)

{
  > <UID:6f1ef041-fdab-4209-adf6-ee8283b7e38b> <UID:🚴🚆> => 🗂️ (StorableObject) (MNIST())
}


In [7]:
model = Net()
params = model.parameters()

In [8]:
# N X C, H, W
# make some data
list_ptr = make([64, 1, 28, 28])
x = torch.ones(list_ptr)
print(x, type(x))

<syft.proxy.torch.TensorPointer object at 0x1360fe2b0> <class 'syft.proxy.torch.TensorPointer'>


In [9]:
model(x)

In [10]:
print(alice.store.peek)

{
  > <UID:6f1ef041-fdab-4209-adf6-ee8283b7e38b> <UID:🚴🚆> => 🗂️ (StorableObject) (MNIST())

  > <UID:b7a4639d-a523-4ecb-aab5-0d47e25ade69> <UID:🚺🚯> => 🗂️ (StorableObject) (MNIST())

  > <UID:0ee71ffc-eb3f-4bb5-9fc3-50713d4509d0> <UID:🙿🚂> => 🗂️ (StorableObject) (<generator object Module.parameters at 0x13605b270>)

  > <UID:0d9720a7-b6bb-45d3-ab41-edd4c93e39e2> <UID:🚅🚶> => 🗂️ (StorableObject) ([64, 1, 28, 28] can_read: ['🚍', '🜡'])

  > <UID:f040f1de-505c-4bac-9423-3f15486fa56f> <UID:🚴🚊> => 🗂️ (StorableObject) (tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1.

In [11]:
params = model.parameters()

In [12]:
# optimizer = optim.Adadelta(params, lr=make(1.0))
# TODO fix ValueError: optimizer got an empty parameter list
# might be incorrect return type for methods and properties

In [13]:
conv1 = nn.Conv2d(make(1), make(32), make(3), make(1))
print(conv1, type(conv1))

<syft.proxy.torch.nn.Conv2dPointer object at 0x136149cd0> <class 'syft.proxy.torch.nn.Conv2dPointer'>


In [14]:
# test all the Functional methods

In [15]:
# N X C, H, W
# make some data
list_ptr2 = make([32, 1, 3, 3])
g = torch.ones(list_ptr)
print(g, type(g))

x = conv1(g)
x = F.relu(x)
x = F.relu(x)
x = F.max_pool2d(x, make(2))
x = torch.flatten(x, make(1))
x = F.relu(x)
output = F.log_softmax(x, dim=make(1))

<syft.proxy.torch.TensorPointer object at 0x136149370> <class 'syft.proxy.torch.TensorPointer'>


In [16]:
# get the output
print(output, type(output))

result = output.get()
print(result)

<syft.proxy.torch.TensorPointer object at 0x1361468b0> <class 'syft.proxy.torch.TensorPointer'>


/Users/madhavajay/dev/PySyft/src/syft/lib/torch/uppercase_tensor.py:54: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  grad = getattr(self.value, "grad", None)


tensor([[-8.8684, -8.8684, -8.8684,  ..., -8.9113, -8.9113, -8.9113],
        [-8.8684, -8.8684, -8.8684,  ..., -8.9113, -8.9113, -8.9113],
        [-8.8684, -8.8684, -8.8684,  ..., -8.9113, -8.9113, -8.9113],
        ...,
        [-8.8684, -8.8684, -8.8684,  ..., -8.9113, -8.9113, -8.9113],
        [-8.8684, -8.8684, -8.8684,  ..., -8.9113, -8.9113, -8.9113],
        [-8.8684, -8.8684, -8.8684,  ..., -8.9113, -8.9113, -8.9113]])


In [17]:
# ORIGINAL MNIST NET
# class Net(nn.Module):
#     def __init__(self) -> None:
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, 1)
#         self.conv2 = nn.Conv2d(32, 64, 3, 1)
#         self.dropout1 = nn.Dropout2d(0.25)
#         self.dropout2 = nn.Dropout2d(0.5)
#         self.fc1 = nn.Linear(9216, 128)
#         self.fc2 = nn.Linear(128, 10)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = F.relu(x)
#         x = self.conv2(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, 2)
#         x = self.dropout1(x)
#         x = torch.flatten(x, 1)
#         x = self.fc1(x)
#         x = F.relu(x)
#         x = self.dropout2(x)
#         x = self.fc2(x)
#         output = F.log_softmax(x, dim=1)
#         return output

# define a Network using make
# class Net(nn.Module):
# # class Net:
#     def __init__(self) -> None:
#         self.conv1 = nn.Conv2d(make(1), make(32), make(3), make(1))
#         self.conv2 = nn.Conv2d(make(32), make(64), make(3), make(1))
#         self.dropout1 = nn.Dropout2d(make(0.25))
#         self.dropout2 = nn.Dropout2d(make(0.5))
#         self.fc1 = nn.Linear(make(9216), make(128))
#         self.fc2 = nn.Linear(make(128), make(10))

#     def forward(self, x):
#         x = self.conv1(x)
#         x = F.relu(x)
#         x = self.conv2(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, make(2))
#         x = self.dropout1(x)
#         x = torch.flatten(x, make(1))
#         x = self.fc1(x)
#         x = F.relu(x)
#         x = self.dropout2(x)
#         x = self.fc2(x)
#         output = F.log_softmax(x, dim=make(1))
#         return output

In [18]:
# ORIGINAL TRAIN
# def train(args, model, device, train_loader, optimizer, epoch):
#     model.train()
#     for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = data.to(device), target.to(device)
#         optimizer.zero_grad()
#         output = model(data)
#         loss = F.nll_loss(output, target)
#         loss.backward()
#         optimizer.step()
#         if batch_idx % args.log_interval == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))
#             if args.dry_run:
#                 break

In [19]:
# ORIGINAL TEST
# def test(model, device, test_loader):
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
#             pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
#             correct += pred.eq(target.view_as(pred)).sum().item()

#     test_loss /= len(test_loader.dataset)

#     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#         test_loss, correct, len(test_loader.dataset),
#         100. * correct / len(test_loader.dataset)))

In [20]:
# fix issue with 
transform_1 = torchvision.transforms.ToTensor()
transform_2 = torchvision.transforms.Normalize(make(0.1307), make(0.3081))

# transform_list = make([transform_1, transform_2])
# print(type(transform_list), transform_list)
# fix cant store pointer? what to do?
# transform = torchvision.transforms.Compose(make([transform_1, transform_2]))

In [21]:
# utility to convert config to pointers
# todo replace with Dict
# need to fix TypeError: 'DictPointer' object is not subscriptable first
def wrap_args_dict(args: dict) -> dict:
    wrapped_args = {}
    for k, v in args.items():
        if sy.lib.python.primitive_factory.isprimitive(v):
            wrapped_args[k] = make(v)
    return wrapped_args

def main():
    # Training settings
    plain_args = {"batch_size":64, "test_batch_size":1000,
            "epochs":14, "lr":1.0, "gamma":0.7,
            "no_cuda":True, "dry_run":False, "seed":42,
            "log_interval":10, "save_model":False}

    args = wrap_args_dict(plain_args)

    use_cuda = not plain_args["no_cuda"] and torch.cuda.is_available()
    device_type_ptr = make("cuda" if use_cuda else "cpu")
    torch.manual_seed(args["seed"])

    device = torch.device(device_type_ptr)
    print("device pointer", device)

    kwargs = {'batch_size': args["batch_size"]}
    if use_cuda:
        kwargs.update(
            wrap_args_dict(
                {
                    'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True
                },
            )
        )
        
    transform_1 = torchvision.transforms.ToTensor()
    transform_2 = torchvision.transforms.Normalize(make(0.1307), make(0.3081))

    # fix cant store pointer? what to do?
    # transform = torchvision.transforms.Compose(make([transform_1, transform_2]))

    dataset1 = datasets.MNIST(make('../data'), train=make(True), download=make(True))
                       #transform=transform)
    dataset2 = datasets.MNIST(make('../data'), train=make(False))
#                        transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **kwargs)

    
    model = Net()
    model = model.to(device)
    # optimizer = optim.Adadelta(model.parameters(), lr=args["lr"])
    # TODO fix ValueError: optimizer got an empty parameter list
    # might be incorrect return type for methods and properties

    # scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
#     for epoch in range(1, args["epochs"].get() + 1):
#         train(args, model, device, train_loader, optimizer, epoch)
#         test(model, device, test_loader)
#         scheduler.step()

#     if args["save_model"]:
#         torch.save(model.state_dict(), "mnist_cnn.pt")

In [22]:
main()

device pointer <syft.proxy.torch.devicePointer object at 0x1365614f0>
<syft.proxy.models.MNISTPointer object at 0x136579af0> <class 'syft.proxy.models.MNISTPointer'>
<syft.proxy.models.MNISTPointer object at 0x136579a90> <class 'syft.proxy.models.MNISTPointer'>
